# Fusion Total

In [ ]:
import tkinter as tk
import threading
import spacy
from collections import Counter

# Charger le modèle de langue français de spaCy
nlp = spacy.load("fr_core_news_sm")

# Fonction pour charger le dictionnaire à partir du fichier texte
def charger_dictionnaire():
    with open("dictionnaire2.txt", "r", encoding="utf-8") as file:
        return [mot.strip() for mot in file]

# Charger le dictionnaire de mots possibles depuis le fichier texte
dictionnaire = charger_dictionnaire()

# Compteur de mots
compteur_mots = Counter(dictionnaire)

# Fonction pour détecter les mots possibles pour le préfixe donné
def detecter_mots_possibles(prefixe):
    mots_possibles = [mot for mot in dictionnaire if mot.startswith(prefixe)]
    # Filtrer les mots possibles commençant par le préfixe et les classer par fréquence d'utilisation
    mots_possibles = sorted(set(mots_possibles), key=lambda x: compteur_mots[x], reverse=True)
    return mots_possibles[:3]  # Sélectionner les 3 premiers mots possibles

# Fonction pour saisir un mot
def saisir_mot():
    mots_actuels = []
    while True:
        lettre = input("Entrez une lettre ou appuyez sur ':' pour enregistrer le mot, ';' pour continuer ou ',' pour quitter : ")
        if lettre == ':':
            mot_saisi = ''.join(mots_actuels)
            # Ajouter le mot saisi au dictionnaire et mettre à jour le compteur de mots
            dictionnaire.append(mot_saisi)
            compteur_mots.update([mot_saisi])
            # Écrire le mot saisi dans le fichier dictionnaire.txt
            with open("dictionnaire2.txt", "a", encoding="utf-8") as file:
                file.write(mot_saisi + "\n")
            return mot_saisi
        elif lettre == ',':
            print("Mots de la phrase:", ' '.join(mots_actuels))
            return "STOP"
        elif lettre == ';':
            continue
        else:
            mots_actuels.append(lettre)
            print("Nouveau mot :")
            print("Le mot actuel est :", ''.join(mots_actuels))
            mots_possibles = detecter_mots_possibles(''.join(mots_actuels))
            if mots_possibles:
                print("Mots possibles :")
                for i, mot in enumerate(mots_possibles):
                    print(f"{i+1}. {mot}")
                while True:
                    choix = input("Appuyez sur Entrer pour continuer ou entrez le numéro du mot pour le sélectionner : ")
                    if choix.isdigit():
                        choix_index = int(choix) - 1
                        if 0 <= choix_index < len(mots_possibles):
                            mots_actuels = [mots_possibles[choix_index]]
                            print(f"Le mot '{mots_possibles[choix_index]}' a été ajouté à votre liste.")
                            # Ajouter le mot au dictionnaire et mettre à jour le compteur de mots
                            dictionnaire.append(mots_possibles[choix_index])
                            compteur_mots.update([mots_possibles[choix_index]])
                            break
                        else:
                            print("Choix invalide. Veuillez entrer un numéro valide.")
                    elif choix == "":
                        break
                    else:
                        print("Veuillez entrer un numéro valide ou appuyez sur Entrée pour continuer.")
            else:
                print("Aucun mot possible trouvé.")
                continue

# Fonction appelée lorsqu'une lettre ou un chiffre est sélectionné
def select_letter(letter):
    entry_var.set(entry_var.get() + letter)
    # Appeler la fonction saisir_mot avec la lettre sélectionnée
    saisir_mot_avec_lettre(letter)

# Ajouter une nouvelle fonction pour traiter la lettre sélectionnée
def saisir_mot_avec_lettre(lettre):
    mots_actuels = [lettre.lower()]
    print("Nouveau mot :")
    print("Le mot actuel est :", lettre.lower())
    mots_possibles = detecter_mots_possibles(lettre.lower())
    
    if mots_possibles:
        print("Mots possibles :")
        for i, mot in enumerate(mots_possibles):
            print(f"{i+1}. {mot}")
        
        while True:
            choix = input("Appuyez sur Entrer pour continuer ou entrez le numéro du mot pour le sélectionner : ")
            
            if choix.isdigit():
                choix_index = int(choix) - 1
                
                if 0 <= choix_index < len(mots_possibles):
                    mot_choisi = mots_possibles[choix_index]
                    print(f"Le mot '{mot_choisi}' a été ajouté à votre liste.")
                    # Ajouter le mot au dictionnaire et mettre à jour le compteur de mots
                    dictionnaire.append(mot_choisi)
                    compteur_mots.update([mot_choisi])
                    return mot_choisi
                else:
                    print("Choix invalide. Veuillez entrer un numéro valide.")
            elif choix == "":
                break
            else:
                print("Veuillez entrer un numéro valide ou appuyez sur Entrée pour continuer.")
    else:
        print("Aucun mot possible trouvé.")
        return None

# Fonction appelée lorsque le bouton "SAVE" est cliqué
def save_word():
    word = entry_var.get()
    print("Mot enregistré :", word)
    # Vous pouvez ajouter ici toute autre fonctionnalité pour enregistrer le mot

# Fonction appelée lorsqu'une touche est pressée
def on_key(event):
    key = event.char
    if key.lower() in 'abcdefghijklmnopqrstuvwxyz0123456789':
        select_letter(key.lower())
    elif event.keysym == "Return":
        saisir_mot_avec_lettre(" ")
        
# Créer la fenêtre principale
root = tk.Tk()
root.title("Clavier Virtuel")

# Cadre principal
frame = tk.Frame(root)
frame.pack(padx=10, pady=10)

# Définir l'espacement supplémentaire entre chaque case
button_padding = {'padx': 20, 'pady': 20}

# Clavier virtuel
keyboard_layout = [
    ('1', '2', '3', '4', '5', '6', '7', '8', '9', '0'),
    ('', '', '', '', '', '', '', '', '', ''),  # Ligne vide pour l'espace
    ('A', 'Z', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P'),
    ('Q', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'M'),
    ('W', 'X', 'C', 'V', 'B', 'N', '', '', '', ''),  # Ligne vide pour l'espace
    ('', '', '', '', '', '', '', '', '', ''),  # Ligne vide pour l'espace
    (',', '.', ';', 'SAVE', 'CLEAR', 'ENTER') # Ajout de la touche "ENTER"
]

# Variable pour stocker le contenu de l'entrée
entry_var = tk.StringVar()

for i, row in enumerate(keyboard_layout):
    for j, key in enumerate(row):
        if key == "":
            # Si la cellule est vide, ajouter un widget transparent
            btn = tk.Label(frame, text='', width=6, height=2)
        elif key == "SAVE":
            btn = tk.Button(frame, text="SAVE", width=6, command=save_word)
        elif key == "CLEAR":
            btn = tk.Button(frame, text="CLEAR", width=6, command=lambda: entry_var.set(""))
        elif key == "ENTER": # Ajout de la gestion de la touche "ENTER"
            btn = tk.Button(frame, text="ENTER", width=6, command=lambda: saisir_mot_avec_lettre(" "))
        else:
            btn = tk.Button(frame, text=key, width=6, command=lambda key=key: select_letter(key))
        btn.grid(row=i, column=j, **button_padding)

# Associer la fonction on_key à l'événement de pression de touche
root.bind('<Key>', on_key)

# Démarrer la saisie de mot en parallèle
t1 = threading.Thread(target=saisir_mot)
t1.start()

# Démarrer l'application
root.mainloop()


# Partie clavier

In [ ]:
import tkinter as tk

# Fonction appelée lorsqu'une lettre ou un chiffre est sélectionné
def select_letter(letter):
    entry_var.set(entry_var.get() + letter)
    print("Lettre ou chiffre sélectionné :", letter)
    # Vous pouvez ajouter ici toute autre fonctionnalité que vous souhaitez exécuter avec la lettre ou le chiffre sélectionné

# Fonction appelée lorsque le bouton "SAVE" est cliqué
def save_word():
    word = entry_var.get()
    print("Mot enregistré :", word)
    # Vous pouvez ajouter ici toute autre fonctionnalité pour enregistrer le mot

# Créer la fenêtre principale
root = tk.Tk()
root.title("Clavier Virtuel")

# Cadre principal
frame = tk.Frame(root)
frame.pack(padx=10, pady=10)

# Définir l'espacement supplémentaire entre chaque case
button_padding = {'padx': 20, 'pady': 20}

# Clavier virtuel
keyboard_layout = [
    ('1', '2', '3', '4', '5', '6', '7', '8', '9', '0'),
    ('', '', '', '', '', '', '', '', '', ''),  # Ligne vide pour l'espace
    ('A', 'Z', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P'),
    ('Q', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'M'),
    ('W', 'X', 'C', 'V', 'B', 'N', '', '', '', ''),  # Ligne vide pour l'espace
    ('', '', '', '', '', '', '', '', '', ''),  # Ligne vide pour l'espace
    (',', '.', ';', 'SAVE', 'CLEAR')
]

# Variable pour stocker le contenu de l'entrée
entry_var = tk.StringVar()

for i, row in enumerate(keyboard_layout):
    for j, key in enumerate(row):
        if key == "":
            # Si la cellule est vide, ajouter un widget transparent
            btn = tk.Label(frame, text='', width=6, height=2)
        elif key == "SAVE":
            btn = tk.Button(frame, text="SAVE", width=6, command=save_word)
        elif key == "CLEAR":
            btn = tk.Button(frame, text="CLEAR", width=6, command=lambda: entry_var.set(""))
        else:
            btn = tk.Button(frame, text=key, width=6, command=lambda key=key: select_letter(key))
        btn.grid(row=i, column=j, **button_padding)

# Démarrer l'application
root.mainloop()


# Partie NLP

Partie dictionnaire

In [ ]:
# Dictionnaire existant
dictionnaire_existant = {
    "chat": 1,
    "chien": 6,
    "maison": 1,
    "voiture": 1,
    "ordinateur": 1,
    "python": 1,
    "code": 1,
    "exemple": 1,
    "possible": 1,
    "cop": 1,
    "co": 5,
    "a": 1
}

# Liste pour stocker les mots en fonction de leurs occurrences
mots_occurrences = []

# Parcourir le dictionnaire existant et ajouter chaque mot autant de fois que son occurrence
for mot, occurrence in dictionnaire_existant.items():
    mots_occurrences.extend([mot] * occurrence)

# Écrire les mots dans un fichier texte
with open("dictionnaire1.txt", "w", encoding="utf-8") as file:
    for mot in mots_occurrences:
        file.write(mot + "\n")

print("Les mots ont été enregistrés dans le fichier 'dictionnaire1.txt'.")


Partie sélection de mot

In [ ]:
import spacy
from collections import Counter

# Charger le modèle de langue français de spaCy
nlp = spacy.load("fr_core_news_sm")

# Fonction pour charger le dictionnaire à partir du fichier texte
def charger_dictionnaire():
    with open("dictionnaire2.txt", "r", encoding="utf-8") as file:
        return [mot.strip() for mot in file]

# Charger le dictionnaire de mots possibles depuis le fichier texte
dictionnaire = charger_dictionnaire()

# Compteur de mots
compteur_mots = Counter(dictionnaire)

# Fonction pour détecter les mots possibles pour le préfixe donné
def detecter_mots_possibles(prefixe):
    mots_possibles = [mot for mot in dictionnaire if mot.startswith(prefixe)]
    # Filtrer les mots possibles commençant par le préfixe et les classer par fréquence d'utilisation
    mots_possibles = sorted(set(mots_possibles), key=lambda x: compteur_mots[x], reverse=True)
    return mots_possibles[:3]  # Sélectionner les 3 premiers mots possibles

# Fonction pour saisir un mot
def saisir_mot():
    mots_actuels = []
    while True:
        lettre = input("Entrez une lettre ou appuyez sur ':' pour enregistrer le mot, ';' pour continuer ou ',' pour quitter : ")
        if lettre == ':':
            mot_saisi = ''.join(mots_actuels)
            # Ajouter le mot saisi au dictionnaire et mettre à jour le compteur de mots
            dictionnaire.append(mot_saisi)
            compteur_mots.update([mot_saisi])
            # Écrire le mot saisi dans le fichier dictionnaire.txt
            with open("dictionnaire2.txt", "a", encoding="utf-8") as file:
                file.write(mot_saisi + "\n")
            return mot_saisi
        elif lettre == ',':
            return "STOP"
        elif lettre == ';':
            continue
        else:
            mots_actuels.append(lettre)
            print("Le mot actuel est :", ''.join(mots_actuels))
            mots_possibles = detecter_mots_possibles(''.join(mots_actuels))
            if mots_possibles:
                print("Mots possibles :")
                for i, mot in enumerate(mots_possibles):
                    print(f"{i+1}. {mot}")
                while True:
                    choix = input("Appuyez sur Entrer pour continuer ou entrez le numéro du mot pour le sélectionner : ")
                    if choix.isdigit():
                        choix_index = int(choix) - 1
                        if 0 <= choix_index < len(mots_possibles):
                            mots_actuels = [mots_possibles[choix_index]]
                            print(f"Le mot '{mots_possibles[choix_index]}' a été ajouté à votre liste.")
                            # Ajouter le mot au dictionnaire et mettre à jour le compteur de mots
                            dictionnaire.append(mots_possibles[choix_index])
                            compteur_mots.update([mots_possibles[choix_index]])
                            break
                        else:
                            print("Choix invalide. Veuillez entrer un numéro valide.")
                    elif choix == "":
                        break
                    else:
                        print("Veuillez entrer un numéro valide ou appuyez sur Entrée pour continuer.")
            else:
                print("Aucun mot possible trouvé.")
                continue

# Boucle principale
mots_enregistres = []
while True:
    print("Nouveau mot :")
    mot = saisir_mot()
    if mot == "STOP":
        break
    elif mot is not None:
        mots_enregistres.append(mot)
        print(f"Le mot '{mot}' a été ajouté à votre liste.")
    else:
        continue

# Concaténer les mots enregistrés avec un espace pour former la phrase
phrase = ' '.join(mots_enregistres)
print("Mots de la phrase:", phrase)
